In [1]:
import os

In [2]:
%pwd

'e:\\Github\\kidney_deep_learning\\Kidney-disease-mlflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Github\\kidney_deep_learning\\Kidney-disease-mlflow'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url : str
    local_data_file: Path
    unzip_dir: Path

In [9]:
%pwd

'e:\\Github\\kidney_deep_learning\\Kidney-disease-mlflow'

In [17]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories


In [18]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
            
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])



    def get_ingestion_config(self) ->DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_url=config.source_url,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )

          return data_ingestion_config
            


In [19]:
import os 
import zipfile
import gdown
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [20]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        """
        Fetch data from the url
        """
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading Data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f"Downloaded Data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """ 
        zip_file_path:str
        Extract the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-14 17:48:48,329:INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-14 17:48:49,957:INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-14 17:48:50,030:INFO: common: created directory at: artifacts]
[2023-10-14 17:48:50,034:INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-14 17:48:50,038:INFO: 3776208973: Downloading Data from https://drive.google.com/file/d/1tiKrxbgquVsl03jiKlOkZQW43CIyr4HJ/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?export=download&id=1tiKrxbgquVsl03jiKlOkZQW43CIyr4HJ
From (redirected): https://drive.google.com/uc?export=download&id=1tiKrxbgquVsl03jiKlOkZQW43CIyr4HJ&confirm=t&uuid=db9b1426-2bf2-4128-a47c-52ec596de832
To: e:\Github\kidney_deep_learning\Kidney-disease-mlflow\artifacts\data_ingestion\data.zip
100%|██████████| 940M/940M [01:24<00:00, 11.1MB/s] 

[2023-10-14 17:50:16,342:INFO: 3776208973: Downloaded Data from https://drive.google.com/file/d/1tiKrxbgquVsl03jiKlOkZQW43CIyr4HJ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
